# Analyzing Toronto Neighborhoods 

## Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

# Part II

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data


## Loading Part I source code from previous notebook

In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests

#Part I
#Loading the dataframe from Wiki page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

#Parsing the HTLM and getting the table
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find('table');

#Parsing the table assigning it to a DataFrame and setting column names
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns = ['PostalCode', 'Borough', 'Neighborhood'])

#Data Cleanup
df['Neighborhood'] = df['Neighborhood'].str.rstrip('\n')
df.drop(0, inplace = True)
df.sort_values(by = 'PostalCode', inplace = True)

#Remove the 'Not assigned' values from Borough column
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)

#Merging rows with duplicate PostalCode and Borough, retain Neighborhood values and separate by comma
df_boroughs = df[['PostalCode', 'Borough']].copy()
df_boroughs.drop_duplicates(keep = 'first', inplace = True)
df_boroughs.reset_index();
df_grouped = df.groupby('PostalCode')['Neighborhood'].apply(', '.join).reset_index()
df_new = pd.merge(df_boroughs, df_grouped, on = 'PostalCode')

#If Neighborhood is 'Not assigned' set it to value of Borough and print postal code M7A for Queen's Park to validate
df_new.loc[df_new.loc[df_new['Neighborhood'] == 'Not assigned'].index,'Neighborhood'] = \
    df_new.loc[df_new['Neighborhood'] == 'Not assigned']['Borough'] 

Getting Latitude and Longitude coordinates from each neighborhood by getting the Postal Code

In [2]:
df_postal_code = df_new[['PostalCode']].copy()
df_postal_code.head(10)

,PostalCode
0,M1B
1,M1C
2,M1E
3,M1G
4,M1H
5,M1J
6,M1K
7,M1L
8,M1M
9,M1N


Getting the Geospatial data

In [3]:
!wget -q 'https://cocl.us/Geospatial_data'

Loading and renaming Postal Code column

In [4]:
df_coords = pd.read_csv('Geospatial_data')
df_coords.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
df_coords.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merging data sorted alphanumerically by PostalCode

In [5]:
df_withcoords = pd.merge(df_new, df_coords, on = 'PostalCode')
df_withcoords.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848
